In [2]:
!pip install keras

In [7]:
import os
import keras
from keras.regularizers import l1,l2,l1_l2

In [4]:
train,test = keras.datasets.cifar10.load_data()


170500096/170498071 [==============================] - 30s 0us/step


In [5]:
def preprocessing(x,flatten=True):
    x = x/255.
    if flatten:
        x = x.reshape(len(x),-1)
    return x

def preprocessing_y(y,num_classes=10):
    if y.shape[-1] ==1:
        y = keras.utils.to_categorical(y,num_classes)
    return y

In [6]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preprocessing(x_train)
x_test = preprocessing(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preprocessing_y(y_train)
y_test = preprocessing_y(y_test)

In [8]:
"""
建立神經網路，並加入 L1 或 L2
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128], l2_ratio=1e-4):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        # first layer of the neural network
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1), 
                                   kernel_regularizer=l2(l2_ratio))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1),
                                   kernel_regularizer=l2(l2_ratio))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model


## hyper parameters setting

LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95
L2_EXP = [1e-2, 1e-4, 1e-8, 1e-12]



results = {}
"""
使用迴圈建立不同的帶不同 L1/L2 的模型並訓練
"""
for regulizer_ratio in L2_EXP:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with Regulizer = %.6f" % (regulizer_ratio))
    model = build_mlp(input_shape=x_train.shape[1:], l2_ratio=regulizer_ratio)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l2-%s" % str(regulizer_ratio)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}
    
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.ylim([0, 5])
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()  


Experiment with Regulizer = 0.010000
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________


W1129 17:46:30.689118 12660 deprecation_wrapper.py:119] From C:\Users\USER\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 5s 93us/step - loss: 15.0525 - accuracy: 0.2682 - val_loss: 13.9272 - val_accuracy: 0.3314
Epoch 2/50
50000/50000 [==============================] - 4s 87us/step - loss: 12.9923 - accuracy: 0.3518 - val_loss: 12.1021 - val_accuracy: 0.3710
Epoch 3/50
50000/50000 [==============================] - 4s 89us/step - loss: 11.3228 - accuracy: 0.3759 - val_loss: 10.5720 - val_accuracy: 0.3799
Epoch 4/50
50000/50000 [==============================] - 4s 87us/step - loss: 9.9118 - accuracy: 0.3900 - val_loss: 9.2747 - val_accuracy: 0.3971
Epoch 5/50
50000/50000 [==============================] - 4s 87us/step - loss: 8.7141 - accuracy: 0.4040 - val_loss: 8.1789 - val_accuracy: 0.4069
Epoch 6/50
50000/50000 [==============================] - 4s 87us/step - loss: 7.6940 - accuracy: 0.4121 - val_loss: 7.2342 - val_accuracy: 0.4168
Epoch 7/50
50000/50000 [==============================] - 4s 8

KeyError: 'acc'